# **Predictions**

In [30]:
import pandas as pd
import numpy as np
import joblib

import warnings
warnings.filterwarnings("ignore")

In [12]:
,# Loading the model
model_filename = "model/logistic_regression_model.pkl"
best_lr_model_2 = joblib.load(model_filename)

Predictions successful


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [15]:
unseen_data = pd.read_csv("model/predictions.csv")

# define the features
X_new = unseen_data

# predictions on the unseen data
predictions = best_lr_model_2.predict(X_new)

# save as CSV file
unseen_data["diabetes_risk"] = predictions  # predictions column with diabetes risk
unseen_data.to_csv("model/predictions_for_unseen_data.csv", index=False)

print("Predictions successful")

Predictions successful


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [26]:
# Function to categorize BMI
def categorize_bmi(bmi):
    if bmi <= 16.9:
        return 2
    elif 17 <= bmi < 18.4:
        return 3
    elif 18.4 <= bmi <= 24.9:
        return 1
    elif 25 <= bmi <= 29.9:
        return 4
    elif 30 <= bmi <= 34.9:
        return 5
    elif 35 <= bmi <= 39.9:
        return 6
    elif bmi >= 40:
        return 7
    else:
        return "Unknown"

# Function to classify blood pressure
def classify_blood_pressure(systolic, diastolic):
    if systolic < 90 and diastolic < 60:
        return 2
    elif systolic > 140 or diastolic > 90:
        return 3
    else:
        return 1

# Function to determine age group
def get_age_group(age):
    bins_age = [18, 25, 35, 45, 55, 65, float("inf")]
    labels_age = ["18-25", "26-35", "36-45", "46-55", "56-65", "Over 65"]
    return pd.cut([age], bins=bins_age, labels=labels_age, right=False)[0]

age_group_mapping = {
    "18-25": 1,
    "26-35": 2,
    "36-45": 3,
    "46-55": 4,
    "56-65": 5,
    "Over 65": 6
}

# Get user input
print("Enter the following medical information:")
gender = input("Gender (Male/Female): ").strip().capitalize()
pulse = float(input("Pulse (in bpm): "))
temp = float(input("Temperature (in Celsius): "))
resp = float(input("Respiratory Rate (in breaths per minute): "))
weight = float(input("Weight (in kg): "))
height = float(input("Height (in cm): "))
age = float(input("Age (in years): "))
systolic = int(input("Systolic Blood Pressure (in mmHg): "))
diastolic = int(input("Diastolic Blood Pressure (in mmHg): "))
is_hypertensive = input("Is the patient hypertensive? (yes/no): ").strip().lower()

# Calculate derived features
bmi = round(weight / ((height / 100) ** 2), 1)
bmi_category = categorize_bmi(bmi)
blood_pressure_category = classify_blood_pressure(systolic, diastolic)
age_group = get_age_group(age)

encoded_age_group = age_group_mapping.get(age_group, 0)

# Prepare the input data for the model
input_data = pd.DataFrame([{
    "age": age,
    "gender": 0 if gender == "Male" else 1,
    "pulse": pulse,
    "temp": temp,
    "resp": resp,
    "bmi_category": bmi_category,
    "age_group": encoded_age_group,
    "blood_pressure_category": blood_pressure_category,
    "is_hypertensive": 1 if is_hypertensive == "yes" else 0
}])

input_data.head()



Enter the following medical information:
Gender (Male/Female): Female
Pulse (in bpm): 67
Temperature (in Celsius): 37
Respiratory Rate (in breaths per minute): 22
Weight (in kg): 64.4
Height (in cm): 160
Age (in years): 35
Systolic Blood Pressure (in mmHg): 135
Diastolic Blood Pressure (in mmHg): 72
Is the patient hypertensive? (yes/no): no


,age,gender,pulse,temp,resp,bmi_category,age_group,blood_pressure_category,is_hypertensive
0,35.0,1,67.0,37.0,22.0,4,3,1,0


In [31]:
# Make predictions
input_data_pred = input_data.drop('age', axis=1)
prediction = best_lr_model_2.predict(input_data_pred)

# Generate output
risk_levels = {0: "Low Risk", 1: "Medium Risk", 2: "High Risk"}
diabetes_risk = risk_levels[prediction[0]]

# Display results
print("\n**Patient Data Summary**")
print(f"Gender: {gender}")
print(f"Pulse: {pulse} bpm")
print(f"Temperature: {temp} °C")
print(f"Respiratory Rate: {resp} breaths/min")
print(f"BMI: {bmi} ({bmi_category})")
print(f"Age Group: {age_group}")
print(f"Blood Pressure Category: {blood_pressure_category}")
print(f"Hypertensive: {'Yes' if is_hypertensive == 'yes' else 'No'}")

print(f"\n**Predicted Risk Level: {diabetes_risk}**")



**Patient Data Summary**
Gender: Female
Pulse: 67.0 bpm
Temperature: 37.0 °C
Respiratory Rate: 22.0 breaths/min
BMI: 25.2 (4)
Age Group: 36-45
Blood Pressure Category: 1
Hypertensive: No

**Predicted Risk Level: High Risk**
